In [31]:
import gradio as gr
from langchain_groq import ChatGroq
from langchain_community.chains.graph_qa.cypher import GraphCypherQAChain
from langchain_community.graphs import Neo4jGraph 
from langchain_community.vectorstores import Neo4jVector
from langchain.prompts import PromptTemplate
from langchain.chains.base import Chain
from langchain.chains import RetrievalQA
from dotenv import load_dotenv
import os
from typing import Any, Dict, List
from langchain.chains.router import RouterChain
from langchain.chains.router.multi_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.llm import LLMChain

In [32]:
load_dotenv()
GROQ_API_KEY = os.environ.get("GROQ_API_KEY")
NEO4J_PASSWORD = os.environ.get("NEO4J_PASSWORD")
NEO4J_URI = os.environ.get("NEO4J_URI")
NEO4J_USERNAME = os.environ.get("NEO4J_USERNAME")
NEO4J_DATABASE = os.environ.get("NEO4J_DATABASE")

In [33]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
hf_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

In [34]:


def create_agentic_rag_components():
    if not GROQ_API_KEY or not NEO4J_PASSWORD:
        raise ValueError("API keys or password not found")
    
    llm = ChatGroq(temperature=0, model = "meta-llama/llama-4-maverick-17b-128e-instruct")

    # --- TOOL 1 ---
    embeddings = hf_embeddings
    vector_store = Neo4jVector.from_existing_index(
        url=NEO4J_URI,
        username=NEO4J_USERNAME,
        password=NEO4J_PASSWORD,
        database=NEO4J_DATABASE,
        embedding=embeddings,
        index_name="monograph_chunks"
    )

    vector_retriever = vector_store.as_retriever()

    vector_qa_chain = RetrievalQA.from_chain_type(
        llm=llm, chain_type="stuff", retriever=vector_retriever, return_source_documents=True
    )
    

    #---TOOL 2---
    graph = Neo4jGraph(
        url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE
    )

    graph.refresh_schema()
    
    graph_qa_chain = GraphCypherQAChain.from_llm(
        llm=llm,
        graph=graph,
        verbose=True,
        return_intermediate_steps=True,
        allow_dangerous_requests=True
    )

    #Descriptions for the Router
    prompt_infos = [
        {
            "name": "vector_search",
            "description": "Good for answering broad, semantic questions or general topics about cancer research.",
            "prompt": PromptTemplate.from_template("{input}")
        },
        {
            "name": "graph_search",
            "description": "Good for answering specific questions about entities and their relationships, like 'What are the leading cancers in a specific city?'.",
            "prompt": PromptTemplate.from_template("{input}")
        },
    ]

    # --- Create the Router Chain ---
    destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
    destinations_str = "\n".join(destinations)
    router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
    router_prompt = PromptTemplate(
        template=router_template,
        input_variables=["input"],
        output_parser=RouterOutputParser(),
    )
    router_chain = LLMRouterChain.from_llm(llm, router_prompt)

    # Create a dictionary to map tool names to their actual chains
    destination_chains = {
        "vector_search": vector_qa_chain,
        "graph_search": graph_qa_chain,
    }

    return router_chain, destination_chains


def query_agentic_rag(question, history):
    if not question or not question.strip():
        return "Please enter a question"
    
    try:
        # 1. Use the router to decide which tool to use
        router_result = router_chain.invoke({"input": question})
        tool_name = router_result['destination']
        
        if tool_name not in destination_chains:
             # Fallback to a default if the router is confused
            tool_name = "vector_search"

        # 2. Get the appropriate chain and run it
        destination_chain = destination_chains[tool_name]
        # Use 'query' for QA chains and 'input' for router
        result = destination_chain.invoke({"query": question})

        answer = result.get("result", "I could not find a definitive answer.")
        details = ""

        if tool_name == "Graph Search":
            intermediate_steps = result.get("intermediate_steps", [])
            cypher_query = intermediate_steps[0].get("query", "N/A") if intermediate_steps else "N/A"
            details = f"**Generated Cypher Query:**\n```cypher\n{cypher_query}\n```"

        elif tool_name == "Vector Search":
            source_docs = result.get("source_documents", [])
            context_preview = "\n\n".join([doc.page_content[:300] + "..." for doc in source_docs[:3]])
            details = (
                f"**Retrieved from Vector Search (Top Chunks):**\n"
                f"> {context_preview.replace(chr(10), chr(10) + '> ')}"
            )   
            

        return (
            f"**Answer:**\n{answer}\n\n"
            f"---\n"
            f"**Tool Used:** {tool_name.replace('_', ' ').title()}\n\n"
            f"{details}"
        )
    
    except Exception as e:
        print(f"Error during query: {e}")
        return (
            f"An error occurred: {e}. "
            f"Please ensure your Neo4j database is running, credentials are correct, and the API is reachable."
        )
    
# Gradio Interface
if __name__ == "__main__":
    print("Initializing Agentic RAG chain...")
    try:
        router_chain, destination_chains = create_agentic_rag_components()
        print("Agentic chain initialized successfully.")

        with gr.Blocks(theme=gr.themes.Soft(), title="Agentic Cancer Monograph RAG") as demo:
            gr.Markdown(
                """
                # 🧠 Agentic GraphRAG Q&A
                This system uses an AI agent to decide whether to use vector search or a knowledge graph query to best answer your question.
                """
            )
            gr.ChatInterface(
                fn=query_agentic_rag,
                title="Ask the Monograph (Agentic)",
                chatbot=gr.Chatbot(height=500),
                textbox=gr.Textbox(placeholder="e.g., What is the link between betel quid chewing and breast cancer?", container=False, scale=7),
                examples=[
                    "What are the leading sites of cancer in Mizoram?",
                    "Tell me about the role of diet in cancer.",
                    "Which studies did the ICMR conduct on tobacco?"
                ]
            )
        
        print("\n🚀 Launching Gradio App... Open the URL in your browser.")
        demo.launch(share=True)
    except Exception as e:
        print(f"Failed to initialize the application: {e}")


Initializing Agentic RAG chain...


d:\Projects\Graph_RAG_&_Hybrid_GraphRAG\env\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Agentic chain initialized successfully.


C:\Users\adity\AppData\Local\Temp\ipykernel_2560\1904678769.py:140: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot=gr.Chatbot(height=500),
d:\Projects\Graph_RAG_&_Hybrid_GraphRAG\env\Lib\site-packages\gradio\chat_interface.py:328: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'tuples', will be used.
  warnings.warn(



🚀 Launching Gradio App... Open the URL in your browser.
* Running on local URL:  http://127.0.0.1:7861

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.
